In [1]:
%reload_ext autoreload
%autoreload 2

This notebook runs part of the GENIES generalisation eval on a model that has adapters

see 
- https://github.com/Joshuaclymer/GENIES
- https://github.com/wassname/GENIES/blob/main/nbs/01_mjc_convert_data_to_preference.ipynb

In [2]:
from datasets import load_dataset

from open_pref_eval.evaluation import evaluate_model
from open_pref_eval.helpers.load_models import load_peft_model

In [3]:
GENIES = [
    {"source": "us_history_textbook", "target": "us_history_fiction", "label": "extreme", "category": "context"},
    {"source": "alpaca_mmlu", "target": "spanish_output", "label": "extreme", "category": "encoding"},
    {"source": "math", "target": "change_my_view", "label": "extreme", "category": "skill"},
    {"source": "raven_matrices", "target": "us_history", "label": "extreme", "category": "skill"},
    {"source": "code_easy", "target": "code_hard", "label": "extreme", "category": "difficulty"},
    {"source": "alpaca_easy", "target": "alpaca_hard", "label": "extreme", "category": "difficulty"},
    {"source": "alpaca_mmlu", "target": "raven_matrices", "label": "extreme", "category": "pretraining_similarity"},
    {"source": "alpaca_mmlu", "target": "ranking_logic", "label": "extreme", "category": "pretraining_similarity"},
    {"source": "alpaca_low_quality", "target": "alpaca_high_quality", "label": "extreme", "category": "quality"},
    {"source": "alpaca_short", "target": "alpaca_long", "target_reference": "alpaca_mmlu", "label": "extreme", "category": "spurious_cues"},
    {"source": "alpaca_mmlu", "target": "wrong_arc", "label": "probing", "category": "spurious_cues"},
    {"source": "alpaca_mmlu", "target": "truthful_qa", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "sycophancy_mimicry", "target_reference": "quote_attribution", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "survival_influence", "label": "probing", "category": "unwanted_personas"},
    {"source": "alpaca_mmlu", "target": "reward_seeking", "label": "probing", "category": "unwanted_personas"}
]

In [4]:
model_name = "gepardzik/LLama-3-8b-rogue-lora"
N = 5000

In [5]:
datasets = []
for row in GENIES:
    name = row['target']
    try:
        ds = load_dataset('wassname/genies_preferences', name=name, split=f'test[:{N}]', keep_in_memory=False)
        datasets.append(ds)
    except ValueError:
        print(f"Dataset {name} not found")
datasets

[Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 250
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 750
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 250
 }),
 Dataset({
     features: ['prompt', 'chosen', 'rejected', 'i'],
     num_rows: 250
 }),
 Dataset({
     featu

In [6]:
model, tokenizer = load_peft_model(model_name)
model

/media/wassname/SGIronWolf/projects5/elk/open_pref_eval/.venv/lib/python3.11/site-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

In [7]:
df_agg, df_raw = evaluate_model(model=model, tokenizer=tokenizer, datasets=datasets,
                                
                                # trl args
                                bf16=True,
                                per_device_eval_batch_size=3,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


  0%|          | 0/15 [00:00<?, ?dataset/s]

KeyError: 'prompt_input_ids'

In [ ]:
from open_pref_eval.plot.radar import radar_plot
df_res = df_raw.groupby(['dataset', 'adapter'], dropna=False)['correct'].mean().unstack()
radar_plot(df_res)
df_res

In [ ]:
df_res

In [ ]:
rename = {}
for row in GENIES:
    s = 'genies_preferences-'+row['target']+'-train'
    rename[s] = row['category']
df_raw['category'] = df_raw.dataset.replace(rename)

In [ ]:
from open_pref_eval.plot.radar import radar_plot
df_res = df_raw.groupby(['category', 'adapter'], dropna=False)['correct'].mean().unstack()
radar_plot(df_res)
df_res